# **Hoja de Trabajo 1 - Visión por Computadora**

- Paula Barillas -
- Gerardo Pineda -
- Mónica Salvatierra - 22249

## **Task 1 - Análisis Teórico**

Considere cada uno de los siguientes escenarios y responda según corresponda:

**1. Como director de un proyecto de conducción autónoma, debe dimensionar el hardware para un
nuevo vehículo. El sistema utiliza 8 cámaras que capturan video a resolución 4K UHD (3840 x 2160).
Debido a la necesidad de alto rango dinámico (HDR), los sensores operan a 12 bits por píxel (Raw
Bayer Pattern) a 60 FPS.Métrica A: Enfocada puramente en el flujo vehicular.**

 **a. Calcule el tamaño exacto de una sola imagen (frame) cruda en Megabytes (MB).**

 **b. Calcule el ancho de banda necesario (en Gbps) para transmitir el flujo de las 8 cámaras al procesador central sin compresión.**

 **c. Si su procesador tiene una memoria RAM reservada de 16 GB exclusivamente para el buffer
de video, ¿cuántos segundos de historia puede almacenar antes de empezar a sobrescribir datos?**

 **d. Basado en su resultado, ¿es viable enviar estos datos "crudos" a la nube en tiempo real
usando 5G? Justifique.**

**2. Considere un píxel con valor de intensidad Iin=50 en una imagen estándar de 8 bits (0−255). Se
aplican dos procesos de mejora secuenciales en el siguiente orden:**

**I. Corrección Gamma con γ=0.5 (para expandir sombras).**

**II. Ajuste Lineal con ganancia α=1.2 y brillo β=-10 (para contrastar).**

Realice los cálculos en el dominio de flotantes normalizados [0,1] como dicta la buena práctica y
convierta a entero de 8 bits solo al final.

 

**3. Usted está programando un robot clasificador de pelotas. Tiene dos objetos: una pelota roja brillante bajo el sol R_rgb=(255,0,0) y la misma pelota roja en una sombra profunda S_rgb=(50,0,0)**

## **Task 2 - Práctica**

## **Task 3 Preguntas Post-Práctica**